In [ ]:
import fiona
import os
import osmnx as ox

## Loading the districting planning area

In [ ]:
# Collect geometries and properties in lists
geometries = []
properties_list = []

with fiona.open('district_and_planning_area.geojson', 'r') as source:
    for feature in source:
        geometries.append(shape(feature['geometry']))
        properties_list.append(feature['properties'])

# Create a GeoDataFrame
gdf_districts = gpd.GeoDataFrame(properties_list, geometry=geometries)

# Plot centers of districts
gdf_districts['centroid'] = gdf_districts['geometry'].centroid


area_exclusion = ["Western Islands", "North-Eastern Islands"]
gdf_districts_filtered = gdf_districts[~gdf_districts["planning_area"].isin(area_exclusion)]

In [ ]:
# Read graph from the saved file
G = ox.load_graphml("singapore_road_network.graphml")

In [ ]:
# Get the nodes as a list
nodes = list(G.nodes(data=True))
nodes[0:2]
nodes.sort(key = lambda x: x[0])
import numpy as np
# Store all the x and y coordinates of the centroids
centroids = np.array([(point.x, point.y) for point in gdf_districts_filtered['centroid']])
# nodes_array = np.array([(data['x'], data['y']) for _, data in nodes])


## Inserting the incinerations nodes

In [ ]:
# Finding the depot node
depot_locations = {"TWTE": (1.3001724, 103.6236303),
                   "KSTP": (1.2976918, 103.6209247),
                   "TSIP": (1.2962427, 103.620556),
                   "SWTE": (1.4632878, 103.794695)}

# Find the nearest node to the depot
depot_nodes_index = [ox.distance.nearest_nodes(G, X=location[1], Y=location[0]) for location in depot_locations.values()]
# Print the depot nodes 
for node_index in depot_nodes_index:
    # Get the node data
    node_data = G.nodes[node_index]
    print(node_data)